In [ ]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':1, 
            'bagging_freq':1,
            'feature_fraction':1, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30.csv',index=False,header=None)

In [30]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7.csv',index=False,header=None)
0.1289

[42.72052106242936, 85.33031787121664, 88.14308354418938]
72.06464082594512
20180302
(26950, 6)


[42.57768517256816, 85.73262233018139, 87.83639659959752]
72.04890136744902
20180303
(27048, 6)


[43.56710461759281, 86.11960185226278, 89.16140508004035]
72.94937051663197
20180304
(27146, 6)


[41.77624791886208, 84.41486020743835, 87.34911963028601]
71.18007591886214
20180305
(27244, 6)


[42.533291677439465, 84.99596718629536, 87.69128104943435]
71.74017997105639
20180306
(27342, 6)


[42.43616926392512, 84.66198984102395, 86.74397722049386]
71.28071210848098
20180307
(27440, 6)


[41.71191152530504, 83.19266033730487, 84.83700445083636]
69.91385877114875
20180308
(27538, 6)


[42.238058475997676, 84.8465150008698, 86.3737428372707]
71.15277210471272
20180309
(27636, 6)


[42.096586636186025, 83.75115401952083, 87.90307180597802]
71.25027082056162
20180310
(27734, 6)


[42.45830444456282, 85.84246756794126, 87.92817056195027]
72.07631419148477
20180311
(27832, 6)


[41.63217932734547,

In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.01, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_0.1.csv',index=False,header=None)
0.1298

[42.40239120706157, 84.842013098729, 87.37445210628047]
71.53961880402368
20180302
(26950, 6)


[42.36413660218462, 84.15106505771202, 86.57347896192371]
71.02956020727345
20180303
(27048, 6)


[42.76873567958814, 84.73206067009802, 87.74100866093504]
71.74726833687373
20180304
(27146, 6)


[41.24132397473044, 82.94828046491858, 86.15143728942164]
70.11368057635688
20180305
(27244, 6)


[42.22300408831153, 83.30458463258454, 86.76266702967392]
70.76341858352333
20180306
(27342, 6)


[42.306013265882925, 83.69015780489148, 85.31727870137415]
70.43781659071618
20180307
(27440, 6)


[41.26731348883131, 82.53969704945034, 83.86915462388725]
69.22538838738963
20180308
(27538, 6)


[41.55296887144685, 84.06305929934135, 85.26457964288196]
70.29353593789006
20180309
(27636, 6)


[41.71813419112691, 83.13178357233535, 85.85730501311502]
70.23574092552576
20180310
(27734, 6)


[42.030588581738435, 84.19546319984718, 87.03910497721104]
71.08838558626555
20180311
(27832, 6)


[41.411860637226646,

In [ ]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20170901].reset_index(drop=True)
print(flow.shape)
cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_3.csv',index=False,header=None)
0.1239

In [4]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20170801].reset_index(drop=True)
print(flow.shape)
cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_2.csv',index=False,header=None)

0.1212

(26852, 6)
(20874, 6)
[44.69751229226826, 85.52046859588376, 88.40208277154342]
72.8733545532318
20180302
(20972, 6)


[44.436189198866145, 83.95356550114914, 86.34951052055303]
71.57975507352278
20180303
(21070, 6)


[43.28489676591276, 85.10010144379773, 87.69912379290403]
72.02804066753818
20180304
(21168, 6)


[44.03714596853278, 84.32026597022043, 86.89150513207815]
71.74963902361046
20180305
(21266, 6)


[43.643955420228494, 85.51398541438121, 88.74355635409503]
72.63383239623491
20180306
(21364, 6)


[43.24747944110648, 84.8308969628551, 86.30285613687104]
71.46041084694421
20180307
(21462, 6)


[43.028022218699434, 84.38232550219519, 86.38083477727515]
71.26372749938993
20180308
(21560, 6)


[42.67594978353502, 83.80482090029884, 85.88846564997682]
70.78974544460355
20180309
(21658, 6)


[42.35489570286889, 82.7612563147383, 85.44982538890545]
70.18865913550421
20180310
(21756, 6)


[42.580926025490264, 83.76197550690145, 84.3052508583198]
70.21605079690384
20180311
(21854, 6)


In [1]:
import datetime
import warnings

import numpy as np
import pandas as pd
from pmdarima.arima import auto_arima

warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)

flow1=flow[flow['city_code']=='3f7f0ce35d6d0a08377eb2efe2189f4f']
flow2=flow[flow['city_code']=='c7537db4101856877ea6381d0174283c']
flow3=flow[flow['city_code']=='58a33c947775af5de36841c9f553317d']
flow4=flow[flow['city_code']=='a20d041605db832309e26c003c626719']
flow5=flow[flow['city_code']=='06d86ef037e4bd311b94467c3320ff38']
flow6=flow[flow['city_code']=='5615dc7c1af1f7dabd80bd8b8ecb1ea0']
flow7=flow[flow['city_code']=='ee2ff207184bf16b4a0aec0f97900c27']
#city1

flow1=flow1[flow1['date_dt']>=20170729]

#city2
flow2=flow2[flow2['date_dt']>=20170729]
#city3
flow3=flow3[flow3['date_dt']>=20170729]
#city4
flow41=flow4[flow4['date_dt']>=20170802]
flow42=flow4[flow4['date_dt']<=20170831]
flow4=flow42.append(flow41)
#city5
flow5=flow5[flow5['date_dt']>=20170729]
#city6
flow6=flow6[flow6['date_dt']>=20170729]
#city7
flow7=flow7[flow7['date_dt']>=20170622]
flow=flow1.append(flow2).append(flow3).append(flow4).append(flow5).append(flow6).append(flow7)
print(flow.shape)
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_2_new.csv',index=False,header=None)


(26852, 6)
(22613, 6)
[48.443694824995944, 95.72122460153128, 99.28385879068404]
81.14959273907041
20180302
(22711, 6)


[47.69378769777235, 95.65709685661162, 99.60514852325683]
80.98534435921361
20180303
(22809, 6)


[48.0164984714954, 96.94638626261367, 100.06395066142953]
81.67561179851288
20180304
(22907, 6)


[48.199764380528755, 94.72337232600601, 99.3189133545924]
80.74735002037572
20180305
(23005, 6)


[47.45864113328841, 94.72360878443239, 97.61805676314987]
79.93343556029022
20180306
(23103, 6)


[47.396842268536815, 93.25740053685608, 97.48794164870299]
79.38072815136529
20180307
(23201, 6)


[47.159520390123085, 95.47147532682652, 99.57173891074179]
80.73424487589712
20180308
(23299, 6)


[46.95237643932451, 94.94725170363475, 98.87553849615499]
80.25838887970475
20180309
(23397, 6)


[46.721689358681274, 94.88223817906238, 98.23155753930295]
79.94516169234886
20180310
(23495, 6)


[46.32719208572318, 93.90487616570826, 97.98986164010064]
79.40730996384401
20180311
(23593,

In [3]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20170901].reset_index(drop=True)
print(flow.shape)
cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])
        
        train_sp = train_x[train_x.columns[2:]]
        test_sp = test_x[test_x.columns[2:]]
        test_x = test_x[test_x.columns[:2]]
        train_x = train_x[train_x.columns[:2]]
        from sklearn.preprocessing import OneHotEncoder
        from scipy import sparse
        enc = OneHotEncoder()
        for feature in [0,1]:
            print(feature)
            enc.fit(train_x[feature].values.reshape(-1, 1))
            train_sp = sparse.hstack((train_sp,enc.transform(train_x[feature].values.reshape(-1, 1)))).tocsr()
            test_sp = sparse.hstack((test_sp,enc.transform(test_x[feature].values.reshape(-1, 1)))).tocsr()

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        print(train_sp.shape)
        print(test_sp.shape)
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_sp[train_part_index.tolist(),:],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_sp[evals_index.tolist(),:],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_sp)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_3_sp.csv',index=False,header=None)

0.1340

(26852, 6)
(17836, 6)
0
1
(13524, 135)
(98, 135)
0
1
(13524, 135)
(98, 135)
0
1
(13524, 135)
(98, 135)
[45.51433287326072, 87.79333449291093, 91.7183489730926]
75.00867211308808
20180302
(17934, 6)


0
1
(13622, 135)
(98, 135)
0
1
(13622, 135)
(98, 135)
0
1
(13622, 135)
(98, 135)
[45.63088377930907, 87.5157269039804, 89.50537632424337]
74.21732900251094
20180303
(18032, 6)


0
1
(13720, 135)
(98, 135)
0
1
(13720, 135)
(98, 135)
0
1
(13720, 135)
(98, 135)
[44.41042295344107, 87.30783303213255, 89.96893638255537]
73.89573078937633
20180304
(18130, 6)


0
1
(13818, 135)
(98, 135)
0
1
(13818, 135)
(98, 135)
0
1
(13818, 135)
(98, 135)
[44.70981631932243, 87.57955993982794, 88.81407096981715]
73.70114907632251
20180305
(18228, 6)


0
1
(13916, 135)
(98, 135)
0
1
(13916, 135)
(98, 135)
0
1
(13916, 135)
(98, 135)
[44.66852505212451, 86.57551030663888, 89.76686197798973]
73.67029911225103
20180306
(18326, 6)


0
1
(14014, 135)
(98, 135)
0
1
(14014, 135)
(98, 135)
0
1
(14014, 135)
(98, 135)
[44.

In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20171001].reset_index(drop=True)
print(flow.shape)
cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_4.csv',index=False,header=None)
1235

(26852, 6)
(14896, 6)
[45.06569658984766, 85.638855650375, 85.94668014803283]
72.21707746275183
20180302
(14994, 6)


[45.98964857567532, 87.45347184540132, 89.2767280775158]
74.23994949953081
20180303
(15092, 6)


[45.37643488497078, 87.33304225180984, 87.29474455498121]
73.3347405639206
20180304
(15190, 6)


[46.4635124564178, 87.62897250164957, 87.93113379771046]
74.00787291859261
20180305
(15288, 6)


[46.36711827234453, 86.26595350134247, 85.27956593844083]
72.63754590404261
20180306
(15386, 6)


[43.53488573811617, 85.8374036498225, 86.06558821016527]
71.81262586603465
20180307
(15484, 6)


[44.14801293645546, 85.57297010549715, 86.1767268756425]
71.96590330586503
20180308
(15582, 6)


[44.96449104196336, 86.48779918584364, 85.02366435774671]
72.15865152851791
20180309
(15680, 6)


[44.240687150877115, 84.93439587584903, 84.71512891249438]
71.2967373130735
20180310
(15778, 6)


[43.670065786964464, 84.20091039778237, 84.17799915042826]
70.6829917783917
20180311
(15876, 6)


[43.7

In [2]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20171101].reset_index(drop=True)
print(flow.shape)
cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_5.csv',index=False,header=None)
0.1344

(26852, 6)
(11858, 6)
[42.94405167757803, 74.63514444122943, 74.17807090436565]
63.919089007724374
20180302
(11956, 6)


[43.14344363400814, 74.59290006579104, 73.4344987709574]
63.72361415691887
20180303
(12054, 6)


[41.78885566949438, 74.80380603897474, 73.7542532198232]
63.448971642764114
20180304
(12152, 6)


[41.184350336604105, 72.8093902173986, 70.22175457782164]
61.40516504394145
20180305
(12250, 6)


[40.44760401172251, 73.79904332158779, 72.54825379123122]
62.26496704151384
20180306
(12348, 6)


[41.056862912787295, 71.60681622396712, 70.4388791540102]
61.03418609692153
20180307
(12446, 6)


[39.77159300289459, 73.05262975449013, 71.40562753601662]
61.40995009780045
20180308
(12544, 6)


[40.71144924866455, 73.92726026808982, 72.6137330372787]
62.41748085134436
20180309
(12642, 6)


[38.07120547454487, 69.84603490499943, 68.77977045982843]
58.89900361312424
20180310
(12740, 6)


[39.577172849624546, 69.80191788093977, 70.27014231831576]
59.88307768296003
20180311
(12838, 6)


In [3]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20171001].reset_index(drop=True)
print(flow.shape)
cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':2,
            'reg_lambda':0, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_4_0.csv',index=False,header=None)
1333

(26852, 6)
(14896, 6)
[44.607399022272205, 83.32459352378865, 82.961369964499]
70.29778750351996
20180302
(14994, 6)


[44.481283952274815, 85.55549742536948, 87.36979851273445]
72.46885996345958
20180303
(15092, 6)


[45.27902772111918, 84.64436112529425, 86.05733117158181]
71.99357333933175
20180304
(15190, 6)


[45.24202711800571, 84.56004899674994, 86.18668219409237]
71.99625276961602
20180305
(15288, 6)


[44.17052924912992, 83.78964643727883, 83.67954011617613]
70.54657193419496
20180306
(15386, 6)


[43.083935301471314, 84.11953673934809, 83.41256596888051]
70.20534600323332
20180307
(15484, 6)


[43.86904621672222, 84.11512999541222, 83.72518138387493]
70.56978586533644
20180308
(15582, 6)


[43.89153643413651, 83.68176400637819, 84.1380990715184]
70.57046650401104
20180309
(15680, 6)


[43.59912774758108, 82.70352787144148, 81.71547001787648]
69.33937521229969
20180310
(15778, 6)


[43.36237150793745, 81.42927654710749, 82.31305379138658]
69.03490061547717
20180311
(15876, 6)


In [4]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
flow = pd.read_csv('data/flow_train.csv')
print(flow.shape)
flow = flow[flow['date_dt']>=20171001].reset_index(drop=True)
print(flow.shape)
cc_se = pd.Series(flow['city_code'].value_counts().index)
dc_se = pd.Series(flow['district_code'].value_counts().index)

flow['city_code'] = flow['city_code'].map(pd.Series(cc_se.index,index=cc_se.values))
flow['district_code'] = flow['district_code'].map(pd.Series(dc_se.index,index=dc_se.values))

for i in range(15):
    rs = []
    labels =['dwell', 'flow_in', 'flow_out']
    for label in labels:
        train = []
        test = []
        for va in flow[['city_code','district_code']].drop_duplicates().values:
            cc = va[0]
            dc = va[1]
            price = flow[(flow['city_code']==cc)&(flow['district_code']==dc)][label].values.tolist()
            k = len(price)
            diff = 30
            l = [cc,dc]
            l.extend(price[k-diff:k])
            l.append(np.nan)
            test.append(l)
            while k-15-diff>=0:
                l = [cc,dc]
                l.extend(price[k-1-diff:k-1])
                l.append(price[k-1])
                k = k-1
                train.append(l)

        test_x = pd.DataFrame(test)
        train_x = pd.DataFrame(train)
        del test_x[train_x.columns[-1]]
        train_y = train_x.pop(train_x.columns[-1])

        params_default_lgb = {
                'num_leaves':80, 
                'learning_rate':0.1, 
            'boosting':'gbdt',
            'min_child_samples':20,

            'bagging_fraction':0.7, 
            'bagging_freq':1,
            'feature_fraction':0.7, 
             'max_depth':-1,
            'reg_alpha':0,
            'reg_lambda':5, 
            'metric':'rmse',
            'objective':'regression'
        }
        NBR = 20000
        VBE = 1000
        ESR = 200
        def get_predict_result(k):
            df = pd.DataFrame()
            num = 0
            rmse = []
            skf = KFold(n_splits=5,random_state=2020,shuffle=True)
            for train_part_index,evals_index in skf.split(train_x,train_y):
                if k==1:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],weight=train_weight.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],weight=train_weight.loc[evals_index],label=train_y.loc[evals_index])
                elif k==0:
                    train_part = lgb.Dataset(train_x.loc[train_part_index],label=train_y.loc[train_part_index])
                    evals = lgb.Dataset(train_x.loc[evals_index],label=train_y.loc[evals_index])
                bst = lgb.train(params_default_lgb,train_part, 
                      num_boost_round=NBR, valid_sets=[train_part,evals], categorical_feature=[0,1],
                      valid_names=['train','evals'],early_stopping_rounds=ESR,
                         verbose_eval=False)
                num+=1
                valid_ypre = bst.predict(test_x)
                df['pred_'+str(num)]=valid_ypre
                rmse.append(bst.best_score['evals']['rmse'])
            return df,rmse
        df,rmse = get_predict_result(0)
        rs.append(np.array(rmse).mean())
        if label=='dwell':
            sub = test_x[[0,1]]
            sub[label] = df.mean(1)
        else:
            sub[label] = df.mean(1)

    print(rs)
    print(np.array(rs).mean())
    sub.columns = ['city_code','district_code', 'dwell', 'flow_in', 'flow_out']
    sub['date_dt'] = 20180302+i
    print(20180302+i)
    sub = sub[flow.columns]
    flow = pd.concat([flow,sub],axis=0,ignore_index=True)
    print(flow.shape)
    print('\n')
flow['city_code'] = flow['city_code'].map(cc_se)
flow['district_code'] = flow['district_code'].map(dc_se)
res = flow[flow['date_dt']>=20180302]
res.to_csv('result/QianYi_30_0.7_4_5.csv',index=False,header=None)
1239

(26852, 6)
(14896, 6)
[44.88392898404427, 85.8815113998841, 86.19281325232069]
72.31941787874969
20180302
(14994, 6)


[45.95346537702399, 87.68821955191984, 89.13649843835086]
74.2593944557649
20180303
(15092, 6)


[45.36966532600406, 87.55758234925187, 87.40532683247791]
73.44419150257795
20180304
(15190, 6)


[46.37986599478326, 86.59230243800026, 87.90270758404748]
73.624958672277
20180305
(15288, 6)


[46.38552438470333, 87.01945143294293, 85.83836411664377]
73.08111331143
20180306
(15386, 6)


[43.77811672488748, 85.81923216451949, 85.5973214396196]
71.7315567763422
20180307
(15484, 6)


[44.08285274928828, 85.43777398027362, 85.48230290918937]
71.6676432129171
20180308
(15582, 6)


[44.80448117387853, 86.34028951304327, 85.71103425363233]
72.28526831351805
20180309
(15680, 6)


[44.016659689594114, 84.51268426257575, 83.8044615128691]
70.77793515501298
20180310
(15778, 6)


[43.85960648798475, 83.52487984653162, 84.24470047780054]
70.5430622707723
20180311
(15876, 6)


[43.28179